# Analyse data collected from the simulator

In [ ]:
import csv
import cv2

lines = []
with open('simulator_images/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
correction = 0.2
images = []
steering_angles = []
for line in lines:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('\\')[-1] #windows in local environment
        current_path = 'simulator_images/IMG/' + filename
        image = cv2.imread(current_path)
        images.append(image)
        if i == 1: #left image
            steering_angle = float(line[3]) + correction
            steering_angles.append(steering_angle)
        if i == 2: #right image
            steering_angle = float(line[3]) - correction
            steering_angles.append(steering_angle)
        if i == 0: #center image
            steering_angle = float(line[3])
            steering_angles.append(steering_angle)
        
    
augmented_images, augmented_steering_angles = [], []
for image, steering_angle in zip(images, steering_angles):
    augmented_images.append(image)
    augmented_steering_angles.append(steering_angle)
    augmented_images.append(cv2.flip(image, 1))
    augmented_steering_angles.append(steering_angle*-1.0)
    

    

# Check size of images collected from training on Udacity Simulator

In [10]:
print(len(steering_angles))
print(len(images))

print(len(augmented_steering_angles))
print(len(augmented_images))

14472
14472
28944
28944


# Transform data into numpy arrays as Keras requires

In [11]:
import numpy as np

#first used images and angles, then augmented
X_train = np.array(augmented_images) # dataset for training
y_train = np.array(augmented_steering_angles) #labels, which in this case are the steering angles

# Build a simple neural networks with keras just to validate that it works

In [5]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=7) #7 epochs

model.save('model.h5')

C:\Users\gusta\Anaconda3\envs\Project3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 3859 samples, validate on 965 samples
Epoch 1/7
3859/3859 [==============================] - 7s 2ms/step - loss: 15498048.1850 - val_loss: 12651.3437
Epoch 2/7
3859/3859 [==============================] - 6s 2ms/step - loss: 7515.4344 - val_loss: 5288.1127
Epoch 3/7
3859/3859 [==============================] - 7s 2ms/step - loss: 4235.9304 - val_loss: 4710.6614
Epoch 4/7
3859/3859 [==============================] - 7s 2ms/step - loss: 3520.3874 - val_loss: 2832.3550
Epoch 5/7
3859/3859 [==============================] - 7s 2ms/step - loss: 2528.1491 - val_loss: 5740.8667
Epoch 6/7
3859/3859 [==============================] - 7s 2ms/step - loss: 2292.3738 - val_loss: 2820.5066
Epoch 7/7
3859/3859 [==============================] - 6s 2ms/step - loss: 2304.2033 - val_loss: 2258.6188


# Preprocess data

In [6]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2) #2 epochs

model.save('model.h5')

C:\Users\gusta\Anaconda3\envs\Project3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 3859 samples, validate on 965 samples
Epoch 1/2
3859/3859 [==============================] - 7s 2ms/step - loss: 2.7471 - val_loss: 0.9103
Epoch 2/2
3859/3859 [==============================] - 7s 2ms/step - loss: 1.8588 - val_loss: 2.9442


# Testing training dataset against LeNet with changes

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout, Flatten, Lambda, Cropping2D
from keras.layers.convolutional import Conv2D, MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((70,25), (0, 0))))
#results into an image of shape 156x316x60
model.add(Conv2D(60, (5,5), activation='relu'))

#results into images of shape 152x312x60
model.add(Conv2D(60, (5,5), activation='relu'))
#pooling layers
#by applying a filter of 2x2, scales down the image to 72x156x60
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#results into an image shape of 68x152x60
model.add(Conv2D(60, (5,5), activation='relu'))
#results into an image shape of 34x76x60
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#results into an image shape of 30x72x30
model.add(Conv2D(30, (5,5), activation='relu'))\
#results into and image shape of 15x36x30
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

#results into an image shape of 11x32x30
#model.add(Conv2D(15, (5,5), activation='relu'))\
#results into and image shape of 6x18x15
#model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.5))

#6x18x15 = 720 nodes
model.add(Flatten())
#model.add(Dense(500))
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, batch_size = 12, shuffle=True, epochs=5)

model.save('model.h5')

Train on 23155 samples, validate on 5789 samples
Epoch 1/5
14208/23155 [=================>............] - ETA: 3:36 - loss: 0.0278

In [12]:
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)